next:

kfold (leave one out?)
look into whitekernel
mess with p value threshold

In [12]:
import distanceCorrelation as dc
import numpy as np
import math
import pandas as pd
import random
import sklearn.gaussian_process as gp
import heapq
import copy
import scipy.stats as sp
import sys
import subprocess
import inspect
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import warnings
import datetime
import networkx as nx


#https://www.geeksforgeeks.org/using-jupyter-notebook-in-virtual-environment/

In [13]:
df = pd.read_excel("input.xlsx")
df.drop(df[df.Group!="Ctrl"].index, inplace=True)
df.drop("Group", axis=1, inplace=True)
print(df)

    Glycerol  2-Hydroxybutyrate  2-Hydroxyisovalerate  3-Hydroxybutyrate  \
0    15669.4              205.2                  56.5              361.5   
1    10084.3              220.9                  92.6              438.0   
2    12836.6              394.7                  93.9              861.2   
3    10520.2              293.0                 200.1             1309.9   
4    15809.9              573.9                 131.3              580.0   
5    12581.0              182.0                  56.8              214.0   
6    12276.4             1103.3                 323.0              422.2   
7    13939.5              488.6                 156.6              493.3   
8    11628.6              567.3                  85.5              377.5   
9    17668.2             1043.2                 178.5              594.8   
10   10293.1              230.4                  52.6             1039.2   
11    6676.8              190.9                  62.1              224.7   
12    8192.4

In [11]:
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

kernel1 = gp.kernels.DotProduct()
kernel2 = gp.kernels.RBF() + gp.kernels.WhiteKernel()
kernel= kernel1+kernel2

#models=[]
loss=[] #to be 2D array for all loss values
count=0.0

for molecule1 in df:
    count+=1
    print(str(round(count*100/len(df.columns),2)) + "% complete")
    #molecule1models=[]
    molecule1Loss=[] #1D array, loss values for single molecule
    for molecule2 in df:
        if molecule1==molecule2: #ensure we don't compare a molecule to itself
            molecule1Loss+=[0.0]
            #molecule1models+=NULL

        else:
            #Splitting into train and validation sets

            sources = df[molecule1].values
            #sources = np.split(x,5)

            for i in range(len(sources)):
                sources[i]=sources.reshape(-1,1)


            targets = df[molecule2]
            #targets = np.split(y,5)

            predictions=[]
            
            for i in range(len(sources)):

                for i in range(5):
                    foldsize=len(sources)//5
                    model = gp.GaussianProcessRegressor(kernel=kernel).fit(sources[0:foldsize*i]+sources[foldsize*i+foldsize:len(sources)],y)
                    #molecule1Loss+=[model.score(cvx,cvy)] #for outputting score

                    #for outputting pvalue
                    predictions+=model.predict(cvx)
                    _,pval=sp.ttest_rel(predictions,cvy)
                    molecule1Loss+=[pval]
            

    loss+=[molecule1Loss]

0.46% complete


ValueError: Found input variables with inconsistent numbers of samples: [30, 24]

In [5]:
#output all correlations above threshold to console

threshold = 0.05

count=0

for i in range(len(loss)):
    for j in range(len(loss[i])):
        if loss[i][j]<threshold:
            print(df.columns[i]+" -> "+df.columns[j]+ " ("+str(loss[i][j])+")")
            count+=1


print(str(count) + " molecules have a score above " + str(threshold))

Glycerol -> Glycerol (0.0)
Glycerol -> ATP (0.0010633879578188165)
Glycerol -> Dimethyl sulfone (0.006641150293867239)
Glycerol -> Succinate (0.0004954945731707063)
Glycerol -> C3-OH (0.000206561112522311)
Glycerol -> C4:1 (0.029264200333412274)
Glycerol -> C5-DC (C6-OH) (0.0021170217567015566)
Glycerol -> C5-M-DC (0.005431288930991725)
Glycerol -> C5:1 (0.0003804860674728835)
Glycerol -> C5:1-DC (0.01488754256438993)
Glycerol -> C6:1 (0.017057887021046512)
Glycerol -> C9 (0.0002515650012196696)
Glycerol -> C10:1 (0.007888028956067186)
Glycerol -> C10:2 (0.019209977177502318)
Glycerol -> C14:1-OH (0.017377572516147747)
Glycerol -> C14:2 (0.001282503439753855)
Glycerol -> C14:2-OH (0.004106102196670345)
Glycerol -> C16-OH (0.010280612958470003)
Glycerol -> C16:2 (0.0013870711646385547)
Glycerol -> C18:1-OH (0.009469890881587202)
Glycerol -> C18:2 (0.022464643209322593)
Glycerol -> Gln (0.03009113479652032)
Glycerol -> Histamine (0.0003611654375575026)
Glycerol -> Carnosine (0.0096910446

In [7]:
#output all correlations above threshold to file

threshold = 0
count=0

filename = ("output_pval/"+kernel1.__class__.__name__+"*"+kernel2.__class__.__name__+"_"+str(datetime.datetime.now())+".csv")
f=open(filename,"w")

for i in range(len(loss)):
    for j in range(len(loss[i])):
        if loss[i][j]>threshold:
            #molecule1, molecule2, correlation score
            #where molecule1 can be used to predict molecule2
            f.write(df.columns[i]+", "+df.columns[j]+ ", "+str(loss[i][j])+"\n")
            count+=1

f.close()

In [33]:

import networkx as nx
df = pd.read_csv("output/float*RBF_2023-03-10 13:41:09.517129.csv")
G = nx.from_pandas_edgelist(df, source="source", target="target")

nx.draw(G)

KeyError: 'target'